In [ ]:
from mpcontribs.client import Client
from pathlib import Path
from pandas import read_csv
import pandas as pd
import numpy as np

In [ ]:
name = "ExpXAS"
client = Client()  # NOTE: set MPCONTRIBS_API_KEY env var

In [ ]:
indir = Path("/Users/patrick/GoogleDriveLBNL/MaterialsProject/gitrepos/mpcontribs-data/ExpXAS")
ref_path = indir / "reference.data"
spec_path = indir / "spectrum.mu"

In [ ]:
# add project-wide meta data to "Other Info" dropdown
client.projects.update_entry(
    pk=name, project={"other": {
        'facility': 'NSLS-II',
        'beamline': 'ISS (8-ID)',
    }}
).result()

In [ ]:
index = "energy [eV]"
ref = read_csv(ref_path, sep=" ", skiprows=[0, 1], index_col=0, names=[index, "reference"])
spec = read_csv(spec_path, sep=" ", skiprows=[0, 1], index_col=0, names=[index, "spectrum"])

In [ ]:
df = pd.concat([ref, spec], axis=1)
df.columns.name = "type"
df.attrs["title"] = "Fe XAS"
df.attrs["labels"] = {"value": "flattened normalized μ(E)"}    
df.attrs["name"] = "Fe-XAS"

In [ ]:
# will probably need to iterate on "data" depending on how a contribution is uniquely identified
# within a project and whether there'll be XAS spectra for multiple elements in the same contribution
contribution = {
    "project": name,
    "identifier": "mp-1279742",  # assign to mp-id or use custom identifier?
    "data": {
        'meta': {
            'year': 2020,
            'cycle': 1,
            'SAF': 304823,
            'proposal': 305112,
            'PI': 'M. Liu',
        },
        'measurement': {
            'method': 'XAS',
            'name': 'FeO',
            'composition': 'Fe',
            'element': 'Fe',
            'edge': 'K',
            'E₀': '7112 eV',  # submit numbers with units as space-separated strings
            'scanID': 77303,
            'UID': 'de753795-be14-402e-9a3f-5089a44ff67c',  # could be linked to BNL raw data
        },
        'time': {
            'start': '01/31/2020  17:20:43',  # TODO parse as datetime
            'stop': '01/31/2020  17:21:44',
            'total': '1 h'
        }        
    },
    "tables": [df]
}

In [ ]:
client.delete_contributions(name)
client.submit_contributions([contribution])

In [ ]:
cid = client.contributions.get_entries(
    project=name, _fields=["id", "identifier"]
).result()["data"][0]["id"]

In [ ]:
contrib = client.get_contribution(cid)

In [ ]:
tid = contrib["tables"][0]["id"]
table = client.get_table(tid)  # pandas DataFrame

In [ ]:
table.display()